In [ ]:
from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import cv2
import numpy as np
import pandas as pd
import random

# open CSV
df = pd.read_csv('METER_ML_train.csv')

#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

print(df.head())

In [ ]:
# Dictionary to track image counts per type
image_counts = {k: 0 for k in ['Landfills', 'RefineriesAndTerminals', 'ProcessingPlants', 'Mines']}

# Function to randomly augment image
def random_augment(image):
    # Define augmentation methods
    methods = [
        lambda img: cv2.flip(img, 1),  # Horizontal flip
        lambda img: cv2.flip(img, 0),  # Vertical flip
        lambda img: cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE),  # Rotate 90 degrees
        lambda img: cv2.cvtColor(cv2.add(cv2.cvtColor(img, cv2.COLOR_RGB2HSV), np.array([15, 25, 0])), cv2.COLOR_HSV2RGB),  # Increase brightness and saturation
        lambda img: (img.astype(np.float32) + np.random.normal(loc=0.0, scale=25.0, size=img.shape)).clip(0, 255).astype(np.uint8)  # Add Gaussian noise
    ]
    # Randomly select an augmentation method and apply it
    return random.choice(methods)(image)

# Function to upload a jpg file to Azure Blob Storage with a given folder name
def upload_jpg_to_blob(image, file_name, folder_name):
    blob_name = f"augmented/{folder_name}/{file_name}"
    blob_client = container_client.get_blob_client(blob_name)
    success, encoded_image = cv2.imencode('.png', image)
    if not success:
        print(f"Error encoding image: {file_name}")
        return
    blob_client.upload_blob(encoded_image.tobytes(), overwrite=True)

# Process images
for index, row in df.iterrows():
    image_type = row['Type']
    if image_type in image_counts:
        if image_counts[image_type] < 10000:
            folder_name = row['Image_Folder']
            file_path = os.path.join(folder_name, row['Image_Folder'].split('/')[-1])  # Adjust based on actual path format in CSV
            
            # Check if the blob exists before downloading
            blob_list = [blob.name for blob in container_client.list_blobs(name_starts_with=file_path)]
            if any(file_path in s for s in blob_list):

                # Download image from Azure Blob Storage
                blob = container_client.download_blob(file_path).readall()
                nparr = np.frombuffer(blob, np.uint8)
                image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
                # Augment the image
                augmented_image = random_augment(image)
            
                # Update the augmented image file name
                augmented_file_name = f"{os.path.basename(file_path)}"
            
                # Upload the augmented image back to blob
                upload_jpg_to_blob(augmented_image, augmented_file_name, folder_name)
            
                # Increment the count for this type
                image_counts[image_type] += 1
            else:
                print(f"Error: File not found in blob storage for path {file_path}")

print("Augmentation process completed.")